In [6]:
# import libraries
import requests
import pandas as pd
from time import sleep

from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import os
import csv

In [7]:
# assign the driver path
driver_path = './geckodriver'

In [8]:
options = webdriver.FirefoxOptions()
options.add_argument('--headless')

In [9]:
# create a driver object using driver_path as a parameter
# driver = webdriver.Chrome(options = options, service = Service(executable_path=driver_path)) # run in background
driver = webdriver.Firefox(service = Service(executable_path=driver_path)) # show browser

In [10]:
# assign your website to scrape
web = 'https://www.amazon.ca'

driver.get(web)

In [11]:
# assign any keyword for searching
keyword = 'Best Seller'

In [12]:
# create WebElement for a search box
search_box = driver.find_element(By.ID, 'twotabsearchtextbox')

In [13]:
# type the keyword in searchbox
search_box.send_keys(keyword)

In [14]:
# create WebElement for a search button 
search_button = driver.find_element(By.ID, 'nav-search-submit-button')

In [15]:
# click search_button
search_button.click()

In [16]:
# wait for the page to download
driver.implicitly_wait(5)

In [17]:
# create empty lists for containing the data we'd like to scrape
product_name = []
product_asin = []
product_price = []
product_ratings = []
product_ratings_num = []
product_link = []
product_img_link = []
results = []


In [18]:
items = driver.find_elements(By.XPATH, '//div[contains(@class, "s-result-item s-asin")]')

In [19]:
items = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "s-result-item s-asin")]')))

In [20]:
for item in items:
    # find product name
    name = item.find_element(By.XPATH, './/span[@class="a-size-base-plus a-color-base a-text-normal"]')
#     product_name.append(name.text)
    
    # find product asin
    data_asin = item.get_attribute("data-asin")
#     product_asin.append(data_asin)
    
    
    # find prices
    whole_price = item.find_elements(By.XPATH, './/span[@class="a-price-whole"]')
    fraction_price = item.find_elements(By.XPATH, './/span[@class="a-price-fraction"]')

    if whole_price != [] and fraction_price != []:
        price = '.'.join([whole_price[0].text, fraction_price[0].text])
    else:
        price = 0
        
#     product_price.append(price)
    
    # find a ratings box
    ratings_box = item.find_elements(By.XPATH, './/div[@class="a-row a-size-small"]/span')

    if ratings_box != []:
        ratings = ratings_box[0].get_attribute('aria-label')
        ratings_num = ratings_box[1].get_attribute('aria-label')
    else:
        ratings, ratings_num = 0, 0

#     product_ratings.append(ratings)
#     product_ratings_num.append(str(ratings_num))

    # find the details link
    link = item.find_element(By.XPATH, './/a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]').get_attribute("href")
#     product_link.append(link)
    
    # find the product image link
    img_link = item.find_element(By.XPATH, './/img[@class="s-image"]').get_attribute('src')
    product_img_link.append(img_link)
    
    results.append((name.text, data_asin, price, ratings, str(ratings_num), link, img_link))

In [21]:
# quit the driver after finishing scraping 
driver.quit()

In [22]:
df = pd.DataFrame(results, columns=["product_name", "product_asin", "product_price", "product_ratings", "product_ratings_num", "product_link", "img_link"])
print(df)

                                         product_name product_asin  \
0   Tu Primera Novela: Como escribir tu Best Selle...   1724272365   
1   Steel Toe Shoes Women Lightweight Air Cushion ...   B08HWWB1SN   
2   Women's Cute Comfy Fuzzy Knitted Memory Foam S...   B073XHFJL3   
3      Mens Hooded Fleece Robe - Plush Long Bathrobes   B075SGYNLT   
4               Best-Seller: Romance (French Edition)   B08R3SMPM9   
5   BEST SELLERS.: LAS HIELES DEL ÉXITO (Spanish E...   B07N8FJXJB   
6                                         Best Seller   1980522065   
7         Women's Cotton Stretch Bikini Panty, 6-Pack   B07CJP12Y3   
8   Squat Proof High Waisted Leggings for Women, T...   B08KQ7TTQM   
9   Vibratorwoman Most Pleasure, Handheld Massage ...   B0B1Y4NV61   
10  Women's Nightgown Pockets Casual V Neck Nights...   B0834MWXS2   
11  Women's Casual Side Bow Tie Neck Short Sleeve ...   B07RK3VHLM   
12  Women's High Waist Flounce Ruffle Top Bikini S...   B07MV5CQG5   
13  Womens Fuzzy Lon

In [23]:
df.to_excel("test.xlsx", sheet_name="test", index=False)

In [24]:
for index, link in enumerate(product_img_link):
    if not os.path.exists("images"):
        os.mkdir("images")
    image = requests.get(link)
    with open("./images" + str(index+1) + ".jpg", "wb") as file:
        file.write(image.content)